In [ ]:
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna
from Bio.SeqUtils import GC
from IPython.display import clear_output
import os


#Parameters to modify
pmin = 20
pmax = 25

gc_min = 40
gc_max = 60

target_tm = 37

min_amp_length = 150
max_amp_length = 200

dGpercentile = 75 #0-100

primer_dimmer_threshold = 0.05 #0-1

#Program variables
v = []
table = []
buff = []
final = []
amplicons = []
buff2 = []

# OPENING CONSENSUS, REMOVING GAPS 
with open("Variation.txt", 'r') as variation:
    with open("consensusSeq.txt", 'r') as infile:
        s = SeqIO.read(infile, "fasta")
        print("Sequence %s found." % (s.id))

        for nc, var in zip(s.seq, variation.readlines()):
            if nc != "-":
                buff.append(nc)
                v.append(float(var.split("\t")[1].split("\n")[0]))

                
s.seq = Seq(''.join(buff), generic_dna)
buff = []

# PRIMER ANALYSIS
for primer_len in range(pmin, pmax):
    #RULE 0: Length
    for si in range(len(s) - primer_len + 1):

        primer = s[si: si + primer_len] 

#           RULE 1: NO GAPS IN THE CONSENSOUS
        if not "-" in primer:

#               RULE 2: mean variation per nucleotide less than 1 & maximun variation less than 100
            vm = sum(v[si: si + primer_len])/primer_len
            if vm < 10 and max(v[si: si + primer_len]) < 100:

#                   RULE 3: GC CONTENT
                gc = GC(primer.seq)
                if gc < gc_max and gc > gc_min:

#                       RULE 4: Tm. ASSUMPTIONS 50 nM primer, 50 mM Na+, and pH 7.0 (http://www.lbgi.fr/MeltingTemperatureBioPhp.php)
                    if primer_len > 13:
                        tm = 64.9 + 41*(primer.seq.count("g") + primer.seq.count("c") - 16.4)/(primer.seq.count("g") + primer.seq.count("c") + primer.seq.count("a") + primer.seq.count("t"))
                    else:
                        tm = 2*(primer.seq.count("a") + primer.seq.count("t")) + 4*(primer.seq.count("g") + primer.seq.count("c"))
                    if tm > target_tm:

                        
                        
                        
#                           RULE 5: No homopolymers of length 4
                        hp = False
                        for i in range(primer_len - 3):   
                            nc = primer.seq[i: i + 4]
                            if (nc[0:2] == nc[2:4]):
                                hp = True
                            
                        if not hp:
                            table.append([si, primer.seq, gc, tm, vm])


# PAIRING PRIMERS // TO-DO, PRIMERS WITH RELATIVELY THE SAME Tm
for f in table:
    for b in table:
        d = b[0]-(f[0]+len(f[1]))
        if d >  min_amp_length and d <  max_amp_length:
            buff.append([f[1],b[1].reverse_complement(),f[0],b[0],s[f[0]:b[0]+len(b[1])]])
        elif d > max_amp_length:
            break
amplicons = buff
buff = []

print("4 first rules and possible amplicons computed. Selected a total of %i amplicons.\n\nComputing dG pre-filtering (LinearFold) analysis..." % (len(amplicons)))

# RULE 5: LinearFold dG filtering
for n,i in zip(amplicons, range(0,len(amplicons))):
    out=os.popen('echo %s | /home/franquero/Documents/LinearFold/linearfold -V' % n[4].seq).read()
    s = float(out.split(" (")[1].split(")")[0])
    buff.append(n + [s])
    clear_output(wait=True)
    print("\nProcessed %i amplicons.\n" % (i))
amplicons = buff
buff = []

#Selectiong the ones inside of the chosen dG percentile
for n in amplicons:
    buff.append(n[5])
    
Glimit = np.percentile(buff,dGpercentile) 
buff = []
print("Percentil %i of dG in the primer sample is %i.\n" % (dGpercentile, Glimit))

for n in amplicons:
    if n[5] > Glimit:
        buff.append(n)
        
amplicons = buff
buff = []

print("dG pre-filtering computed. Selected a total of %i amplicons.\n\nComputing primer dimmer analysis..." % (len(amplicons)))

# RULE 6: NUPACK // PRIMER DIMER & REGION NOT STRUCTURED
        

for amp in amplicons:
    with open ("temp.in", "w") as infile:
        infile.write("2" + '\n' + ('\n'.join([str(amp[0]) ,str(amp[1])])) + '\n' + "1 2" + '\n')
        infile.close()

    cmd_input = str.encode("temp")
    p = sub.Popen(["pairs", "-multi","-material", "dna", "temp"], stdin=sub.PIPE, stdout=sub.PIPE, stderr=sub.STDOUT)
    stdout = p.communicate()

    with open("temp.ppairs", "r") as temp:   
        outputlines = temp.readlines()
        temp.close()
        os.remove("temp.ppairs")
        os.remove("temp.epairs")
        os.remove("temp.in")

    for line in outputlines:
        if len(line.split("\t")) > 1 and line[0] != '\n' in line:
            if len(buff) == 0:
                buff = np.array([[float(i) for i in line.split("\t")[0:3]]])
            else:
                buff = np.append(buff, [[float(i) for i in line.split("\t")[0:3]]],0)

    score = 0
    for n in buff:
        nc = n[0] - len(amp[0]) 
        if -10 < nc <= 0:
            nc = n[1] - len(amp[0]) 
            if 0 < nc <= 10:
                k = 1 + ((5 - nc) * 0.2)
                if score == 0:
                    score = (score + (k * n[2]))
                else:
                    score = (score + (k * n[2]))/2
                #print(str(k) + " " + str(n[2]) + " " + str(score))

        nc = n[0] - len(amp[0])        
        if 0 < nc <= 10:
            nc = n[1] - len(amp[0]) 
            if -10 < nc <= 0:
                k = 1 + ((5 + nc) * 0.2)
                if score == 0:
                    score = (score + (k * n[2]))
                else:
                    score = (score + (k * n[2]))/2
                #print(str(k) + " " + str(n[2]) + " " + str(score))

    if  score < primer_dimmer_threshold:
            amp.append(score)
            buff2.append(amp)
        
    buff = []
    

    
amplicons = buff2
buff2 = []



print("Primer dimmer analysis computed. Selected a total of %i amplicons.\n\nComputing amplicon structure analysis..." % (len(amplicons)))

#NuPack amplicon analysis

#FINAL LIST OF PRIMERS FOR BLAST
for n in table:
    include = False
    for amp in amplicons:
        if n[1] == amp[0] or n[1].reverse_complement() == amp[1]:
            include = True
    if include:
        buff.append(n)
        
table = buff
buff = []

print("%i total primers for %i amplicons.\n\nComputing BLASTn analysis." % (len(table),len(amplicons)))

#RULE 7: BLAST

